In [33]:
#imports
import numpy as np
import heapq

In [34]:
#load small input
input = open("./../../data/day_17/small_input.txt", "r").readlines()

In [52]:
#load small2 input
input = open("./../../data/day_17/small2_input.txt", "r").readlines()

In [55]:
#load input
input = open("./../../data/day_17/input.txt", "r").readlines()

In [21]:
# a node in this case is a tuple of (x,y,v,ed,si) where x,y is the position, v is the value (heat loss), ed is the enter direction and si is the straight line already used up 
# directions: 0 = right 1 = down 2 = left 3 = up
shape = (len(input),len(input[0].strip()))
values = np.zeros(shape,dtype=int)
nodes = []
for i,line in enumerate(input):
    for j,char in enumerate(line.strip()):
        value = int(char)
        for si in range(3):
            for d in range(4):
                nodes.append((i,j,value,d,si))
        values[i,j] = value

def in_arr(pos, shape):
    x,y = pos
    return x >= 0 and y >= 0 and x < shape[0] and y < shape[1]

def get_neighbours(node):
    x,y,v,ed,si = node
    neighbours = []
    pos_dirs = []
    if si < 2:
        pos_dirs.append(ed) #direction entered can be continued as long as current straight line is not already 3 blocks
    pos_dirs.append((ed+1)%4) # reverse direction is not possible, but 90 degrees left and right is possible
    pos_dirs.append((ed+3)%4) # same as above

    for d in pos_dirs:
        match d:
            case 0: #right
                p = (x,y+1)
            case 1: #down
                p = (x+1,y)
            case 2: #left
                p = (x,y-1)
            case 3: #up
                p = (x-1,y)
        if in_arr(p, shape):
            neighbours.append( (p[0],p[1],values[p],d, si + 1 if ed==d else 0) )

    return neighbours




In [22]:
# part 1
## dijkstra
## initialize dijkstra
distance = np.full((len(nodes)), 1e6, dtype=int)
visited = np.full((len(nodes)), False, dtype=bool)
distance[0] = 0
prev = np.full((len(nodes)), -1, dtype=int)

q = nodes.copy()
cache_indices = {}
for i,n in enumerate(q):
    cache_indices[n] = i
last_left_to_visit = 1e6
while np.sum(visited == False) > 0 and np.sum(visited==False) != last_left_to_visit:
    if last_left_to_visit % 1000 == 0:
        print(f"There are {last_left_to_visit} nodes left to visit")
    # print(f"left to visit {np.sum(visited==False)}")
    last_left_to_visit = np.sum(visited==False)
    tdist = np.copy(distance)
    tdist[visited] = 1e6
    # print(f"dist {np.min(distance)}")
    # print(f"tdist {np.min(tdist)}")
    # print(f"visited total {np.sum(visited)}")
    index_u = np.argmin(tdist)
    # print(f"Iterating Dijkstra with node {index_u}")
    visited[index_u] = True
    u = q[index_u]
    neighbours = get_neighbours(u)
    for v in neighbours:
        index_v = cache_indices[v]
        if not visited[index_v]:
            alt = distance[index_u] + v[2] #v[2] is the value of entering the block, which is the cost of the edge
            if alt < distance[index_v]:
                distance[index_v] = alt
                prev[index_v] = index_u

# extract solution
target_nodes = []
for node in nodes:
    if node[0] == shape[0]-1 and node[1] == shape[1]-1:
        target_nodes.append(node)

shortest_target_distance = 1e10
for target in target_nodes:
    index = nodes.index(target)
    shortest_target_distance = min(shortest_target_distance, distance[index])

print(f"Part 1: {shortest_target_distance}")


There are 1000000.0 nodes left to visit
There are 2000 nodes left to visit
There are 1000 nodes left to visit
Part 1: 102


In [56]:
#part 2
##redefine node loading and neighbour function
# a node in this case is a tuple of (x,y,v,ed,si) where x,y is the position, v is the value (heat loss), ed is the enter direction and si is the straight line already used up 
# directions: 0 = right 1 = down 2 = left 3 = up
shape = (len(input),len(input[0].strip()))
values = np.zeros(shape,dtype=int)
nodes = []
#manually add the node for the start position which has no current straight line
for i,line in enumerate(input):
    for j,char in enumerate(line.strip()):
        value = int(char)
        for si in range(10):
            for d in range(4):
                nodes.append((i,j,value,d,si))
        values[i,j] = value


def in_arr(pos, shape):
    x,y = pos
    return x >= 0 and y >= 0 and x < shape[0] and y < shape[1]

def get_neighbours(node):
    x,y,v,ed,si = node
    neighbours = []
    pos_dirs = []
    if si < 9:
        pos_dirs.append(ed) #direction entered can be continued as long as current straight line is not already 3 blocks
    if si >= 3: #only allow turning after moving at least four
        pos_dirs.append((ed+1)%4) # reverse direction is not possible, but 90 degrees left and right is possible
        pos_dirs.append((ed+3)%4) # same as above

    for d in pos_dirs:
        match d:
            case 0: #right
                p = (x,y+1)
            case 1: #down
                p = (x+1,y)
            case 2: #left
                p = (x,y-1)
            case 3: #up
                p = (x-1,y)
        if in_arr(p, shape):
            neighbours.append( (p[0],p[1],values[p],d, si + 1 if ed==d else 0) )

    return neighbours




In [57]:
# part 2

## dijkstra
def dijkstra(nodes, start_node):
    distances = {node: 1e6 for node in nodes}
    distances[start_node] = 0
    prev = {node:None for node in nodes}

    q = [(0,start_node)]

    cache_neighbours = {}

    i = 0
    while len(q) > 0:
        if i % 1000 == 0:
            print(f"Iteration {i}")
        i += 1

        cur_dist, cur_node = heapq.heappop(q)
        # if cur_dist > distances[cur_node]:
        #     continue

        neighbours = cache_neighbours.get(u, get_neighbours(cur_node))
        if u not in cache_neighbours:
            cache_neighbours[cur_node] = neighbours
        for v in neighbours:
            alt = cur_dist + v[2] #v[2] is the value of entering the block, which is the cost of the edge
            if alt < distances[v]:
                distances[v] = alt
                prev[v] = cur_node
                heapq.heappush(q, (alt,v))
    return distances

# get start nodes and run
#set distance for start positions
value_start = nodes[0][2]
start_right = (0,4,value_start,0,3)
start_down = (4,0,value_start,1,3)

distance_right = dijkstra(nodes, start_right)
distance_down = dijkstra(nodes, start_down)

# extract solution
target_nodes = []
for node in nodes:
    if node[0] == shape[0]-1 and node[1] == shape[1]-1 and node[4] >= 3:
        target_nodes.append(node)

distance_start_right = np.sum(values[0,1:5])
distance_start_down = np.sum(values[1:5,:])
shortest_target_distance = 1e10
for target in target_nodes:
    shortest_target_distance = min(shortest_target_distance, distance_down[target]+distance_start_down, distance_right[target]+distance_start_right)

print(f"Part 2: {shortest_target_distance}")


Iteration 0
Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000
Iteration 10000
Iteration 11000
Iteration 12000
Iteration 13000
Iteration 14000
Iteration 15000
Iteration 16000
Iteration 17000
Iteration 18000
Iteration 19000
Iteration 20000
Iteration 21000
Iteration 22000
Iteration 23000
Iteration 24000
Iteration 25000
Iteration 26000
Iteration 27000
Iteration 28000
Iteration 29000
Iteration 30000
Iteration 31000
Iteration 32000
Iteration 33000
Iteration 34000
Iteration 35000
Iteration 36000
Iteration 37000
Iteration 38000
Iteration 39000
Iteration 40000
Iteration 41000
Iteration 42000
Iteration 43000
Iteration 44000
Iteration 45000
Iteration 46000
Iteration 47000
Iteration 48000
Iteration 49000
Iteration 50000
Iteration 51000
Iteration 52000
Iteration 53000
Iteration 54000
Iteration 55000
Iteration 56000
Iteration 57000
Iteration 58000
Iteration 59000
Iteration 60000
Iteration 61000
Iteration 62000
Itera